Selecting data structures
=========================

The following example, is based on some real-world code, but cleaned up a little to illustrate the concepts more clearly.  The data is an extract of [citation data](http://snap.stanford.edu/data/cit-HepTh.html) from [arxiv.org](arxive.org)'s High Energy Physics papers made available by the SNAP project.  They were originally pepared for the [2003 KDD cup](http://www.cs.cornell.edu/projects/kddcup/).

The data is stored in a file with a tab-separated format like this:

    # FromNodeId	ToNodeId
    1001	9304045
    1001	9308122
    1001	9309097
    1001	9311042
    ...

The first column stores the id of the paper, and the second the id of a paper cited by that paper.

For more information about this data set, see:

- J. Leskovec, J. Kleinberg and C. Faloutsos. Graphs over Time: Densification Laws, Shrinking Diameters and Possible Explanations. ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2005.

- J. Gehrke, P. Ginsparg, J. M. Kleinberg. Overview of the 2003 KDD Cup. SIGKDD Explorations 5(2): 149-151, 2003.

## Creating an inverse citation index

The code here uses Pandas to read in the table into a data frame, and then iterates over it, building a reverse citation index in a dictionary (ie. for a given paper, which papers is it cited by?)



In [1]:
import pandas as pd
import time

# get the CPU time at the start of reading data in
t = time.clock()

# read in the data
df = pd.read_table('cit-HepTh.txt', sep='\t', comment='#', names=['paper', 'citation'])

# create an empty dictionary for the reverse citation index
cited_dict = {}

# for each row in the data frame
for nline in xrange(0, df.shape[0]):
    # get the id of the paper and cited paper
    paper = df['paper'][nline]
    citation = df['citation'][nline]

    # if we haven't seen the cited paper before, add it to our index
    if citation not in cited_dict.keys():
        cited_dict[citation] = []

    # add the citing paper to the list
    cited_dict[citation].append(paper)

print 'Time taken:', time.clock()-t

print "Paper id 1001 was cited by:", cited_dict.get(1001, 0)

Time taken: 173.704626
Paper id 1001 was cited by: [5068, 7170, 7195, 11075, 104044, 105207, 112101, 209230, 212114, 212223]


This is very slow.  The data set is large, but not really huge: it has 27770 papers, with 352807 citation records. 
3 minutes is too slow for such a small data set!

In [2]:
# number of papers cited by some other paper
print "Number of papers cited by some other paper:", len(cited_dict)

Number of papers cited by some other paper: 23180


So *why* does it take too long?

The main reason that there is a problem is this line:

    if citation not in cited_dict.keys():

It seems like it should be straightforward, but when executed it:

* `cited_dict.keys()` creates a new list containing the keys of the dictionary (which we just saw ends up with 23180 elements, so it's not small).
* `not in` then linearly searches that list for a match.

Fortunately, you can use `not in` with just the dictionary to get the same result without creating a new list, and with an O(1) search instead of a linear search.

In [3]:
t = time.clock()

# read in the data
df = pd.read_table('cit-HepTh.txt', sep='\t', comment='#', names=['paper', 'citation'])

# create an empty dictionary for the reverse citation index
cited_dict = {}

# for each row in the data frame
for nline in xrange(0, df.shape[0]):
    # get the id of the paper and cited paper
    paper = df['paper'][nline]
    citation = df['citation'][nline]

    # if we haven't seen the cited paper before, add it to our index
    if citation not in cited_dict:
        cited_dict[citation] = []

    # add the citing paper to the list
    cited_dict[citation].append(paper)

print 'Time taken:', time.clock()-t

print "Paper id 1001 was cited by:", cited_dict.get(1001, 0)

Time taken: 8.594614
Paper id 1001 was cited by: [5068, 7170, 7195, 11075, 104044, 105207, 112101, 209230, 212114, 212223]


Just doing that gives a massive speedup.

There are a few other tricks which can help as well, like the `setdefault()` method of dictionaries and extracting the values from the data frame (which gives a numpy array) and iterating over them directly. 

In [4]:
t = time.clock()

# read in the data
df = pd.read_table('cit-HepTh.txt', sep='\t', comment='#', names=['paper', 'citation'])

# create an empty dictionary for the reverse citation index
cited_dict = {}

# for each row in the data frame
papers = df['paper'].values
citations = df['citation'].values
for nline in xrange(df.shape[0]):
    citing_papers = cited_dict.setdefault(citations[nline], list())
    citing_papers.append(papers[nline])

print 'Time taken:', time.clock()-t

print "Paper id 1001 was cited by:", cited_dict.get(1001, 0)

Time taken: 0.639104
Paper id 1001 was cited by: [5068, 7170, 7195, 11075, 104044, 105207, 112101, 209230, 212114, 212223]


This is realistically about as fast as you are likely to be able to get it to go.

## Efficient queries

The choice of data structures is also affected by the kind of queries we are planning to do on them. In this case, for instance, we might want to ask questions like: "Given a list of 'query' papers, find the papers that were cited by all of them."

In [5]:
t = time.clock()

query = [5068, 7195]
cited_by_all = []

# For each entry in the dictionary...
for cited, citing in cited_dict.items():
    # ... verify if all the papers in `query` are among the citing papers.
    for paper in query:
        # If not, break out of the loop.
        if paper not in citing:
            break
    else:
        cited_by_all.append(cited)

        
print 'Time taken:', time.clock()-t
print 'Papers cited by all the papers in the query:', cited_by_all

Time taken: 0.23832
Papers cited by all the papers in the query: [1001, 9610237, 9506144, 9507158]


In this case, we see that choosing lists for storing the citing papers is not optimal: we do not care about the order of the papers, and verifying if a paper is part of the list is O(N).

If we store the citing papers in a set, instead, the same operation is only O(1). Even better, we can check if the set of query papers  is a subset of the set of citing papers, saving the for loop:

In [6]:
# Load the data, as in the previous examples.
cited_dict = {}
papers = df['paper'].values
citations = df['citation'].values
for nline in xrange(0, df.shape[0]):
    # The citing papers are accumulated in a set, not a list.
    citing_papers = cited_dict.setdefault(citations[nline], set())
    citing_papers.add(papers[nline])

t = time.clock()

query = set([5068, 7195])
cited_by_all = set()

# For each entry in the dictionary...
for cited, citing in cited_dict.items():
    # ... verify if all the papers in `query` are among the citing papers.
    if query <= citing:
        cited_by_all.add(cited)
        
print 'Time taken:', time.clock()-t
print cited_by_all

Time taken: 0.014856
set([9506144, 1001, 9610237, 9507158])


## Line-by-line file processing [optional]

Finally, pandas data reading routines are very fast, but it does have the problem that it loads all the data into memory at once.

If you do it in pure Python, without using pandas, you can still do it pretty quickly, but with the advantage that it doesn't use as much memory:

In [7]:
t = time.clock()

# create an empty dictionary for the reverse citation index
cited_dict = {}

# open the file for reading
with open('cit-HepTh.txt') as fp:
    for line in fp:
        # skip comments
        if line.startswith('#'):
            continue
        
        # get data from the line
        paper, citation = line.split()
        
        # convert strings to ints (not strictly needed)
        paper = int(paper)
        citation = int(citation)
        
        # append the paper to the list of citing papers (creating it if needed)
        citing_papers = cited_dict.setdefault(citation, set())
        citing_papers.add(paper)

print 'Time taken:', time.clock()-t

print "Paper id 1001 was cited by:", cited_dict.get(1001, 0)

Time taken: 1.256692
Paper id 1001 was cited by: set([7170, 11075, 112101, 5068, 104044, 209230, 212114, 105207, 7195, 212223])
